# Feedback Delay Network (FDN) Reverb

FDN reverb simulates complex acoustic spaces by feeding the output of multiple delay lines back into their inputs through a mixing matrix

- Delay Lines: Store the audio signal for different, specific lengths of time
- Feedback Matrix: A grid (matrix) that mixes and routes the output of every delay line back to the input of other delay lines
- Mixing: Control how much dry signal enters the delay lines and how the final wet signal from the network is combined
- Filter: digital filters to dampen and make the reverb sound more natural

In use here is https://en.wikipedia.org/wiki/Hadamard_matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display
from scipy.io import wavfile
import warnings

# Parameters
n_delays = 4
delay_lengths = [1051, 1249, 1399, 1597]
damping = 0.5
feedback_gain = 0.5
wet = 0.8

# Load audio file
audio_path = "../../resources/drums1.wav"
warnings.simplefilter("ignore", wavfile.WavFileWarning)
sr, y = wavfile.read(audio_path)

# Handle stereo by taking mean
if y.ndim == 2:
    y = y.mean(axis=1)

# Normalize to -1 to 1 range
waveform = y.astype(np.float32) / np.max(np.abs(y))

# Ensure waveform is 1D
waveform = waveform.flatten()

# Pad with silence to hear reverb tail
waveform = np.concatenate([waveform, np.zeros(sr * 2)])

# Initialize delay buffers
delay_buffers = [np.zeros(length) for length in delay_lengths]
buffer_indices = [0] * n_delays
last_output = np.zeros(n_delays)

# Create Hadamard feedback matrix
H2 = np.array([[1, 1], [1, -1]])
feedback_matrix = np.block([[H2, H2], [H2, -H2]]) / 2

# Process audio through FDN reverb
output = np.zeros_like(waveform)

for i, sample in enumerate(waveform):
    # Read from delay lines
    delayed_samples = np.array([delay_buffers[d][buffer_indices[d]] for d in range(n_delays)])
    
    # Apply damping (lowpass filter)
    delayed_samples = damping * delayed_samples + (1 - damping) * last_output
    last_output = delayed_samples.copy()
    
    # Apply feedback matrix
    feedback_signals = feedback_matrix @ delayed_samples
    feedback_signals *= feedback_gain
    
    # Write to delay lines
    for d in range(n_delays):
        delay_buffers[d][buffer_indices[d]] = sample + feedback_signals[d]
        buffer_indices[d] = (buffer_indices[d] + 1) % delay_lengths[d]
    
    # Mix wet and dry
    reverb_out = np.sum(delayed_samples) / n_delays
    output[i] = (1 - wet) * sample + wet * reverb_out

# Visualize
plt.figure(figsize=(12, 4))
plt.plot(waveform[:30000], label='Dry', alpha=0.7)
plt.plot(output[:30000], label='With Reverb', alpha=0.7)
plt.title('Audio: Dry vs With Reverb')
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.legend()
plt.tight_layout()
plt.show()

print("Dry signal:")
display(Audio(waveform, rate=sr))

print("With reverb:")
display(Audio(output, rate=sr))